# Group #3
- Beatrice Nobile
- Bertrand Leclercq
- Théophile Tolani

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from io import StringIO
from csv import writer
import datetime

# Import data
Import the data and create a Pandas data frame with it.

In [ ]:
directory = "../input/ecommerce-behavior-data-from-multi-category-store/2019-Oct.csv"
directory2 = "../input/ecommerce-behavior-data-from-multi-category-store/2019-Nov.csv"

october = pd.read_csv(directory, header='infer', parse_dates=['event_time'], date_parser=pd.to_datetime, nrows=1)
november = pd.read_csv(directory2, header='infer', parse_dates=['event_time'], date_parser=pd.to_datetime)
data = pd.concat([october, november], ignore_index=True)

data.head()

In [ ]:
data.shape

---

# Question 1

***A marketing funnel describes your customer’s journey with your e-commerce. It may involve different stages, beginning when someone learns about your business, when he/she visits your website for the first time, to the purchasing stage, marketing funnels map routes to conversion and beyond. Suppose your funnel involves just three simple steps: 1) view, 2) cart, 3) purchase. Which is the rate of complete funnels?***

Since we don't need all the columns, we are only going to consider the ones we care about, namely [event_type, product_id, user_id]. Once we have these, we're probably going to have a lot of duplicates, since each user could have viewed, added to cart or purchased a product more than once, but what we are interested in is only if they have undergone the complete process, so we can drop all of these duplicates. Once we have the cleaned dataset, we can see whether each combination of [product_id, user_id] has all three event types or not and compute the rate of complete funnels.

In [ ]:
# Select the data we need and clean it 
sub_df = data.loc[:, ["event_type", "product_id", "user_id"]]
tot_events = sub_df.drop_duplicates().shape[0]
sub_df = sub_df.drop_duplicates().groupby(['product_id', 'user_id'])

complete_funnels = 0

# Check if all three events have occurred for each [product_id, user_id] combination
for idx, group1 in sub_df:
    if len(group1) == 3:
        complete_funnels += 1

rate_complete_funnels = round(complete_funnels/tot_events, 3)

print("The rate of complete funnels is:", rate_complete_funnels)

As we go on, since the dataset is so big and we will be creating sub-datasets in order to avoid working with the whole of it, and also to filter our data according to our needs, we are going to delete anything we create as soon as we use it, so as to free up memory usage.

In [ ]:
lst = [october, november, sub_df]
del october, november, sub_df
del lst

# Question 1.1

***What’s the operation users repeat more on average within a session? Produce a plot that shows the average number of times users perform each operation (view/removefromchart etc etc).***

In [ ]:
# We count each event type per each session
event_per_session = pd.DataFrame(data.value_counts(subset = ["user_session", "event_type"], 
                                                   sort = False), columns = ["count"])
event_per_session.reset_index(level=["event_type"], inplace=True)

# We then groupby event type and compute the mean
averages = event_per_session.groupby(by = "event_type", sort = False).mean().to_dict()
averages = averages["count"]

# Plot
plt.bar(*zip(*averages.items()), width= 0.6, color = ["darkblue", "blue", "lightblue"])
plt.xlabel("Event Types", fontdict={'fontsize': 14}) 
plt.ylabel("Averages", fontdict={'fontsize': 14}) 
plt.title("Events' Occurrences per Session", fontdict={'fontsize': 16})
plt.savefig("Events' Occurrences per Session")
plt.show()

In [ ]:
del event_per_session

# Question 1.2

***How many times, on average, a user views a product before adding it to the cart?***

First of all, we need to take into consideration the fact that a user might view the product even after adding it to cart or even after purchasing it. Below we see an example:

In [ ]:
data.loc[(data["product_id"] == 5100816) & (data["user_id"] == 550121407)]

Since we need to understand how many times the user views the product **before** adding it to the cart, we must count only those occurrences that come before that event. We first select only the columns we are interested in, and create two groups, one for the "view" event, and one for the "cart" event. Then we compare the two groups, merge them on the base of the product_id and the user_id and then subtract from the resulting dataframe the rows that come after the time when the product was added to the cart. Finally, we count those rows.

In [ ]:
# We consider only those columns we need and for future questions, we convert event_time
# from string to datetime.
sub_df = data.loc[:, ["event_time", "event_type", "product_id", "user_id"]]

# We create the groups we need 
cart_products = sub_df.query('event_type=="cart"').product_id

group_cart = sub_df.query('event_type=="cart"').groupby(['user_id', 'product_id'])

group_view = (sub_df[sub_df.product_id.isin(cart_products)]
              .query('event_type=="view"').groupby(['user_id', 'product_id']))

# We retrieve the views for each [product_id, user_id] combination that has added 
# to cart the product. Once we have them, we consider only those views that occurred
# before adding to cart.
counts = []
for idx, group1 in group_cart:
    try:
        group2 = group_view.get_group(idx)
    except KeyError:
        # handle rare case of a cart event with no view event
        counts.append(0)
        continue
    counts.append(group2.loc[group2.event_time.lt(group1.event_time.iloc[0]), 'event_type'].count() )

result_1_2 = round(sum(counts)/len(counts), 2)
print(f"On average an user views the product {result_1_2} times before adding it to the cart.")

In [ ]:
lst = [cart_products, group_cart, group_view]
del cart_products, group_cart, group_view
del lst

# Question 1.3

***What’s the probability that products added once to the cart are effectively bought?***

We already have the sub_dataframe with only the columns we are interested on (sub_df). We also already know which are the products that have been added to the cart by users (cart_products). Based on this group of products, we can see which are the products that have been purchased after being carted. Then, we use exactly the same method as before, but now, we check for what has appened after adding the product to the cart. Another difference with respect to before is that we are not really interested in knowing how many times they purchase the product, but only if they do.

In the mean time, we prepare the ground for question 1.4, and we extract all the information on those customers that have added to the cart a certain product, but have not proceeded into purchasing it.

In [ ]:
group_purchased = (sub_df[sub_df.product_id.isin(cart_products)]
              .query('event_type=="purchase"').groupby(['user_id', 'product_id']))

bought = []

# The following method is the fastest way we have found to create the subgroups we need.
# In this case, all those products that have been added to cart, and subsequently purchased
# or removed from cart.
output = StringIO()
csv_writer = writer(output)

for idx, group1 in group_cart:
    try:
        group2 = group_purchased.get_group(idx)
    except KeyError:
        # these are the cases of 'cart' events without 'purchase' events 
        bought.append(0)
        csv_writer.writerow(group1.to_numpy()[0])
    if group2.loc[group2.event_time.gt(group1.event_time.iloc[0]), 'event_type'].count() > 0:
        bought.append(1)
        
output.seek(0) 
removed = pd.read_csv(output, header=None, sep='\n')
removed = removed[0].str.split(',', expand=True)
removed.columns = ["event_time", "event_type", "product_id", "user_id"]
removed[["product_id", "user_id"]] = removed[["product_id", "user_id"]].apply(pd.to_numeric)

result_1_3 = round(sum(bought)/len(bought), 2)

print("The probability of purchasing a product after having added it to the cart is:", result_1_3)

In [ ]:
del group_purchased

# Question 1.4

***What’s the average time an item stays in the cart before being removed?***

The first thing we notice here is that the dataset doesn't actually contain "removefromcart" event. Yet, as suggested by the author of the dataset, we can figure out removed from cart products as the difference between purchase and cart events. 

In [ ]:
data.loc[data["event_type"] == "removefromcart"]

Again, we can't use aggregate functions, because we need to consider the timing. A product can be classified as "removed from cart" only after it has been added to cart and afterwards it has not been purchased, and that is what we have built in the "removed" dataframe.

In [ ]:
removed

Quick check to see if indeed the combinations ["product_id", "user_id"] in the removed dataframe are indeed those that have added the products on the cart but not purchased them afterwards.

In [ ]:
data.loc[(data["product_id"] == 4700419) & (data["user_id"] == 107837897)]

So now we know which products that had been added to the cart have not afterwards been purchased, and hence that at some point must have been removed from the cart. To compute how much time the product has stayed in the cart, we can look at the last time the user has viewed the product, after having added it to the cart, and once he/she has not viewed it anymore, then we suppose that they have removed it from the cart. For those that do not view the product after the cart, we assume that the element has immediately been removed from the cart.

Therefore, we now need to retreive the last time the product has been viewed for each [user_id, product_id] pair that has been classified as "removed_from_cart".

In [ ]:
# We collect the data we need with the conditions we need, and in particular 
# only the last views for each [product_id, user_id] combination.

removed_products = list(removed.product_id)

group_removed = removed.groupby(['product_id', 'user_id'])

group_view = (sub_df[sub_df.product_id.isin(removed_products)]
              .query('event_type=="view"').groupby(['user_id', 'product_id']).last())
group_view = group_view.groupby(['product_id', 'user_id'])

# We use the same method to get the subgroup of those products that have been removed
# from cart and then viewed.
output = StringIO()
csv_writer = writer(output)

for idx, group1 in group_removed:
    try:
        group2 = group_view.get_group(idx)
        csv_writer.writerow(group2.to_numpy()[0])
    except KeyError:
        # cases where the combination [product_id, user_id] has not been removed
        continue
    
output.seek(0) 
last_viewed = pd.read_csv(output, header=None, sep='\n')
last_viewed = last_viewed[0].str.split(',', expand=True)
last_viewed.columns = ['user_id', "product_id", "event_time", "event_type"]
last_viewed[["product_id", "user_id"]] = last_viewed[["product_id", "user_id"]].apply(pd.to_numeric)

In [ ]:
last_viewed.head()

Once we know also the last view for each [product_id, user_id] combination where the product has been removed from the cart, we can then concatenate the two groups and take the time difference for each occurrence and then compute the average.

In [ ]:
# Now the filtered data can be combined and grouped.
cart_last_view = pd.concat([removed, last_viewed])
g_cart_last_view = cart_last_view.groupby(["product_id", "user_id"])

time_in_cart = []

# For each [product_id, user_id] combination, we look for the time of the last view
# and when the user had added to the cart the product. If the user's last view is
# before the cart event, we assume the product has been immediately removed. 
for idx, group1 in group_removed:
    try:
        group2 = g_cart_last_view.get_group(idx)
        if list(group2.event_time)[1] > list(group2.event_time)[0]:
            time_in_cart.append(((pd.Timedelta((list(group2.event_time)[1] #Timedelta.value is in nanoseconds
                                               - list(group2.event_time)[0])).value)/1000000000))
        else:
            time_in_cart.append(0)
    except IndexError:
        # cases where the product has not been viewed 
        time_in_cart.append(0)
        
        
result_1_4 = sum(time_in_cart) / len(time_in_cart)

sec = datetime.timedelta(seconds = result_1_4)
d = datetime.datetime(1,1,1) + sec

print(f"On average before being removed a product stays in the cart:")
print("%d Day and %d:%d:%d" % (d.day-1, d.hour, d.minute, d.second))

In [ ]:
lst = [removed, group_removed, group_view, last_viewed, cart_last_view, g_cart_last_view]
del removed, group_removed, group_view, last_viewed, cart_last_view, g_cart_last_view
del lst

# Question 1.5

***How much time passes on average between the first view time and a purchase/addition to cart?***

To answer this question, we first create three different dataframes that are grouped by the [product_id, user_id] combination, and that contain only the first occurrence of each event type. Indeed, a user may buy a product more than once, may also add it to cart, then remove it, and then add it again, etc...

Once we have these subgroups, we can compute the time difference between each event's first occurrence.

In [ ]:
# Filter the date according to needs: get the first occurrence for each event.
first_view = (sub_df.query('event_type=="view"').groupby(['user_id', 'product_id']).first()).groupby(['user_id', 'product_id'])
first_cart = (sub_df.query('event_type=="cart"').groupby(['user_id', 'product_id']).first()).groupby(['user_id', 'product_id'])
first_purchase = (sub_df.query('event_type=="purchase"').groupby(['user_id', 'product_id']).first()).groupby(['user_id', 'product_id'])

In [ ]:
# Compute time difference between first view and cart event.
time_cart = []

for idx, group1 in first_cart:
    try:
        group2 = first_view.get_group(idx)
        if list(group1.event_time)[0] > list(group2.event_time)[0]:
            time_cart.append((pd.Timedelta((list(group1.event_time)[0] 
                                            - list(group2.event_time)[0])).value)/1000000000) 
    except:
        continue

        
result_1_5_1 = sum(time_cart) / len(time_cart)

sec_1 = datetime.timedelta(seconds = result_1_5_1)
d_1 = datetime.datetime(1,1,1) + sec_1

print(f"On average, the time it takes a user to add a product to the cart after the first view is:")
print("%d Days and %d:%d:%d" % (d_1.day-1, d_1.hour, d_1.minute, d_1.second))

In [ ]:
# Compute time difference between first view and purchase event.
time_purchase = []

for idx, group1 in first_purchase:
    try:
        group2 = first_view.get_group(idx)
        if list(group1.event_time)[0] > list(group2.event_time)[0]:
            time_purchase.append((pd.Timedelta((list(group1.event_time)[0] 
                                            - list(group2.event_time)[0])).value)/1000000000)
    except:
        continue
        

result_1_5_2 = sum(time_purchase) / len(time_purchase)

sec_2 = datetime.timedelta(seconds = result_1_5_2)
d_2 = datetime.datetime(1,1,1) + sec_2

print(f"On average, the time it takes a user to purchase a product after the first view is:")
print("%d Days and %d:%d:%d" % (d_2.day-1, d_2.hour, d_2.minute, d_2.second))

In [ ]:
lst = [first_view, first_cart, first_purchase, sub_df]
del first_view, first_cart, first_purchase, sub_df
del lst

---

# Question 2

***What are the categories of the most trending products overall? For each month visualize this information through a plot showing the number of sold products per category.***

For this question, we need only the data from the sold products, which mean the data where the event_type is 'purchase'. For the category, we use the category_code which is the meaningful name of the category (in the format category.subcategory, so we apply a function to extract only the category name). The best visualisation for this data is the "bar" plot.

In [ ]:
# Use only the data from purchase (= sold products) 
purchases = data[data.event_type == 'purchase'].copy()

# Extract the category name from the category_code attribute
purchases['category_code'] = purchases['category_code'].apply(lambda x: str(x).split('.')[0])

purchasesByMonth = purchases.groupby([purchases.event_time.dt.month])

# Group the data by month and for each month, make a plot
for month, frame in purchasesByMonth:
    
    categoriesCount = frame['category_code'].value_counts()
    
    x = list(map(str, categoriesCount.index))
    heigths = categoriesCount.values
    
    plt.figure(figsize=(21,7))
    plt.bar(x, heigths)
    plt.title(f'Month #{month}')
    plt.xticks(rotation=90)
    plt.show()
    
del purchases

# Question 2.1

***Plot the most visited subcategories.***

This is similar from the previous question, except that we need the visits and not the sold data (so the data where the event_type is 'view') and we want all subcategories (and not only categories).

In [ ]:
# Use only the data from view (= visited products) 
visits = data[data.event_type == 'view'].copy()

# Group the data by month and for each month, make a plot
for month, frame in visits.groupby([visits.event_time.dt.month]):
    
    categoriesCount = frame['category_code'].value_counts()
    
    x = list(map(str, categoriesCount.index))
    heigths = categoriesCount.values
    
    plt.figure(figsize=(21,7))
    plt.bar(x, heigths)
    plt.title(f'Month #{month}')
    plt.xticks(rotation=90)
    plt.show()
    
# Delete variables that will not be used in next questions
del visits

# Question 2.2

***What are the 10 most sold products per category?***

For this question, I reuse the `purchasesByMonth` variable from the question 2. For each month, we group the data by category and then for each category, we counts the number of product and print the 10 higher number whith the id of the product.

In [ ]:
for month, frame in purchasesByMonth:
    
    print(f'Month {month}')
    print('----------------------------------')
    
    for categoryCode, frame2 in frame.groupby([frame.category_code]):
        
        print(f'Categorie: {categoryCode}')
        print(frame2['product_id'].value_counts().head(10))
        print()

# Delete variables that will not be used in next questions
del purchasesByMonth

---


# Question 3

***For each category, what’s the brand whose prices are higher on average?***

# Question 3.1

***Write a function that asks the user a category in input and returns a plot indicating the average price of the products sold by the brand.***

For this question, we fist extract the category name from the category_code attribute, print all possible category and wait for an input from the user (which need to be in the list of category). Then, we extract the data from the given category, group the category data by brand and compute the price mean before plotting the result in a 'bar' plot (which is the more suitable for this visualisation).

In [ ]:
dataCopy = data.copy()

# Extract the category name from the category_code attribute
dataCopy['category_code'] = dataCopy['category_code'].apply(lambda x: str(x).split('.')[0])    

# Ask the user for a category and check if it exists
while True:
    print(f'Enter a category ({", ".join(dataCopy["category_code"].unique())}):')
    category = input()
    if category in dataCopy["category_code"].unique():
        break

# Extract the data from the given category
categoryData = dataCopy[dataCopy.category_code == category]
# Group the category data by brand and compute the price mean
brandPriceAverage = categoryData.groupby([categoryData.brand]).price.mean()

# Plot the price mean / brand
x = brandPriceAverage.index
heigths = brandPriceAverage.values

plt.figure(figsize=(21,7))
plt.bar(x, heigths)
plt.title(f'Average price by brand for "{category}" products')
plt.xticks(rotation=90)
plt.show()

# Delete variables that will not be used in next questions
del category, categoryData, brandPriceAverage, x, heigths

# Question 3.2

***Find, for each category, the brand with the highest average price. Return all the results in ascending order by price.***

For this question, we group the data by category and by brand and we compute the price mean. The for each category, we apply a sort_value to sort the brand and their mean price in descending order and get the first one (thus the brand with the highest mean price). After that, we sort the category in ascending order.

In [ ]:
dataCopy = dataCopy.groupby([dataCopy.category_code, dataCopy.brand]).price.mean()
dataCopy.groupby('category_code', group_keys=False).apply(lambda x: x.sort_values(ascending=False).head(1)).sort_values()

In [ ]:
# Delete variables that will not be used in next questions
del dataCopy

---

# Question 4

***How much does each brand earn per month? Write a function that given the name of a brand in input returns, for each month, its profit. Is the average price of products of different brands significantly different?***

In [ ]:
# Again, we select only those columns we need, and since we want to know about profits
# we will only consider the purchase event. Moreover, since we don't care about the time
# of purchase, but only about its month, we have changed the format of the dates.
sub_df = data.loc[:, ["event_time", "event_type", 
                      "brand", "price"]].query("event_type == 'purchase'")
sub_df['event_time'] = pd.to_datetime(sub_df["event_time"].str.slice(0, -13), format = '%Y-%m-%d')
sub_df = sub_df.set_index(pd.DatetimeIndex(sub_df['event_time']))
sub_df.drop(['event_time', 'event_type'], axis = 1, inplace = True)

# We group by months with grouper, and then group by brands to take the mean price
sub_df = pd.DataFrame(sub_df.groupby([pd.Grouper(freq='M'), 'brand'])['price'].mean())
sub_df.reset_index(level='brand', inplace = True)

# We take notice of those brands that are in the dataset, but do not seem to make a profit
# in the two months considered.
brands = list(sub_df.brand.drop_duplicates())
all_brands = list(data.brand.drop_duplicates())
if len(brands) != len(all_brands):
    missing_brands = [i for i in all_brands if i not in brands]

# Since we will need to know if the month is October or November, we thought it easier
# and more understandable to change it again into a string and replace the value with 
# the months' names
sub_df.reset_index(inplace = True)
sub_df['event_time'] = sub_df['event_time'].dt.strftime('%Y-%m-%d')
sub_df['event_time'] = sub_df.replace({'event_time': {'2019-10-31': 'october', '2019-11-30': 'november'}})
g_sub_df = sub_df.groupby('brand')

In [ ]:
def brand_monthly_profit(brand):
    group = g_sub_df.get_group(brand)
    if brand in brands:
        profits = {}
        for i in range(len(group)):
            profits[list(group.event_time)[i]] = list(group.price)[i]
        return profits
    else:
        print(f"{brand}'s profits for October and November are 0")

In [ ]:
brand_monthly_profit('aqua')

In [ ]:
mean = round(sub_df.price.mean(), 2)
mini = round(sub_df.price.min(), 2)
maxi = round(sub_df.price.max(), 2)
var = round(sub_df.price.var(), 2)

print("The average profits change greatly among brands, as the variance shows.")
print(f"In fact, the variance is {var}, and indeed while the mean is {mean}, we have maximum value {maxi} and minimum {mini}.")

# Question 4.1

***Using the function you just created, find the top 3 brands that have suffered the biggest losses in earnings between one month and the next, specifing both the loss percentage and the 2 months (e.g., brand_1 lost 20% between march and april).***

In [ ]:
# We consider only those brands that actually made a profit in both months
losses = {}
worst_brands = [0, 0, 0]
brand_names = ['', '', '']

for i in brands:
    profits = brand_monthly_profit(i)
    if len(profits) == 2: # checking if they made a profit in both months
        if profits['october'] > profits['november']: #checking if they made a loss 
            
            # We immediately compare the results and save only those 
            #  3 that feature the biggest losses.
            if (profits['october'] - profits['november']) > min(worst_brands):
                idx = worst_brands.index(min(worst_brands))
                worst_brands[idx] = profits['october'] - profits['november']
                brand_names[idx] = i
                losses[i] = round((profits['november'] 
                                   - profits['october'])/profits['october']*100, 2)    
    else:
        continue

for bran in brand_names:
    print(f"{bran} lost {losses[bran]}% between October and November.")

---

# Question 5

***In what part of the day is your store most visited? Knowing which days of the week or even which hours of the day shoppers are likely to visit your online store and make a purchase may help you improve your strategies. Create a plot that for each day of the week show the hourly average of visitors your store has.***

##### Solution 1
Solved using .groupby() to split our data. In order to know how many people are accessing the website I decided to rely on the number of user sessions accessing the website. We first split by day in order to count our data day per day. Then we count how many user sessions are accessing the website and group them for each hour. After that we only have to plot the results for each day on one single graph. We can see that there are clearly two attendance peaks : one around 8am in the morning and one around 5pm in the afternoon.

In [ ]:
# First Group by Day
for group in data.groupby([data.event_time.dt.day]):
    
    #Then Group by hours for each day and count each user_session
    dfp = group[1].groupby([data.event_time.dt.hour]).user_session.count()
    
    #Plot
    dfp.plot()
    plt.title(label = 'Number of visits per hour')
    plt.show()

##### Solution 2

Solved using a .resample() method instead of .groupby(). Works with the same logic as the previous solution and comes to the same conclusions.

In [ ]:
df = data[['event_time','product_id','event_type']]

df = df.set_index(['event_time','product_id']).count(level="event_time")

df.resample('H').mean().plot()

---

# Question 6

***The conversion rate of a product is given by the number of times a product has been bought over the number of times it has been visited. What's the conversion rate of your online store?***

# Question 6.1

***Find the overall conversion rate of your store.***

# Question 6.2

***Plot the number of purchases of each category and show the conversion rate of each category in decreasing order.***

In [ ]:
a = len(data[data['event_type']=='view'])
print('The website has been visited '+str(a)+' times.')
b = len(data[data['event_type']=='purchase'])
print('There were '+str(b)+' purchases.')
print('The overall conversion rate of the store is then :', str(b/a))

In [ ]:
purch = data[data.event_type == 'purchase'].groupby('category_id').count()
purch = purch[['event_type']]
plt.scatter(np.linspace(0,len(purch),len(purch)),purch)
plt.title('Number of purchases for each category')
plt.show()

In [ ]:
purch.head(5)

In [ ]:
views = data[data.event_type == 'view'].groupby('category_id').count()
views = views[['event_type']]

In [ ]:
L = []

# By using the length of the dataframe 'purch' (which contains all the purchases for each category), 
# we drop every category with no purchase.These have a conversion rate of 0. 
for k in range(len(purch)):
    L.append(int(purch.iloc[k])/int(views.iloc[k]))

L.sort(reverse = True)

purch['Conversion Rate'] = L

In [ ]:
print('Conversion Rate of each category in decreasing order')
purch.head(10)

---

# Question 7

***The Pareto principle states that for many outcomes roughly 80% of consequences come from 20% of the causes. Also known as 80/20 rule, in e-commerce simply means that most of your business, around 80%, likely comes from about 20% of your customers. Prove that the pareto principle applies to your store.***

In [ ]:
sub_df = data.loc[:, ["event_type", "brand", "price"]].query("event_type == 'purchase'")
all_customers = set(sub_df.brand.dropna())
sales = pd.DataFrame(sub_df.groupby('brand')['price'].sum()).sort_values(by= "price", 
                                                                         ascending = False)
sales.head()

In [ ]:
percentage_customer = int(0.2*len(all_customers)) #543
tot_sales = int(sales.sum()) #485584186
target_sales = int(0.8*tot_sales) #388467348
sales_20perc = int(sales[0:percentage_customer].sum()) #482125166
sales_generated_20perc = round(sales_20perc/tot_sales*100, 2) #99.29%

In [ ]:
if sales_20perc > target_sales:
    print(f"20% of the online store customers create {sales_generated_20perc}% of the store's total sales.")

In [ ]:
sub_df = data.loc[:, ["event_type", "user_id", "price"]].query("event_type == 'purchase'")
all_customers = set(sub_df.user_id.dropna())
sales = pd.DataFrame(sub_df.groupby('user_id')['price'].sum()).sort_values(by= "price", 
                                                                         ascending = False)
sales.head()

In [ ]:
percentage_customer = int(0.2*len(all_customers)) #139494
tot_sales = int(sales.sum()) #505152392
target_sales = int(0.8*tot_sales) #404121913
sales_20perc = int(sales[0:percentage_customer].sum()) #364222233
sales_generated_20perc = round(sales_20perc/tot_sales*100, 2) #72.1%

In [ ]:
print(percentage_customer, tot_sales, target_sales, sales_20perc, sales_generated_20perc)

In [ ]:
if sales_20perc > target_sales:
    print(f"20% of the online store customers create {sales_generated_20perc}% of the store's total sales.")